# 종목 정보 덤프

In [5]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 1. 종목정보덤프
print(">> 1. DUMP cap >= 500000000000 & KOSPI")

# 1.0. 기존 데이터 제거
print("1.0. Delete old data")
cursor.execute("TRUNCATE cap")
cursor.execute("TRUNCATE stocks_price")
db.commit()

# 1.1 시가총액 업데이트
print("1.1. Update cap")
# 1.1.1. 매수종목조회
print("1.1.1. Get already bid ticker")
cursor.execute("SELECT ticker FROM bid_basket")
basketTickers = list(map(lambda x: x[0], cursor.fetchall()))
basketTickers.remove('test')
# 1.1.2. 최신데이터call
print("1.1.2. Get new data")
capDf = stock.get_market_cap_by_ticker(str(datetime.now().date()).replace('-', ''), market='KOSPI')
capDf.index.names = ['ticker']
capDf = capDf.rename(columns={'종가':'close', '시가총액':'cap'}).drop(['거래량', '거래대금', '상장주식수'], axis=1)
basketDf = capDf.loc[basketTickers].copy()
capDf = capDf[capDf['cap'] >= 500000000000]
totalDf = pd.concat([basketDf,capDf]).drop_duplicates()
db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
conn = db_connection.connect()
totalDf.to_sql(name='cap', con=db_connection, if_exists='append', index=True, index_label="ticker")
conn.close()

# 1.2. 티커리스트 조회
print("1.2. Get ticker list")
db.commit()
cursor.execute("SELECT ticker FROM cap")
tickers = list(map(lambda x: x[0], cursor.fetchall()))
print(tickers)
db.close()

# 1.3. 기간 설정
print("1.3. Set period")
start = '20210601'
end = str(datetime.now().date()).replace('-', '')
print("period: (%s - %s)" %(start, end))

# 1.4. 새로운 데이터 업데이트
print("1.4. Update new data (%s - %s)" %(start, end))
for ticker in tickers:
  try:
    df = stock.get_market_ohlcv_by_date(start, end, ticker)
    time.sleep(0.1)
    df['ticker'] = ticker
    df = df.rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})    
    df.index.names = ['date']
    df = df[['ticker', 'open', 'high', 'low', 'close', 'volume']]

    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
    conn.close()
  except Exception as ex:
    print(ex, ticker)
    pass

# 1.5. 거래이상 종목제거
print("1.5. Delete outlier")
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("DELETE FROM stocks_price WHERE ticker IN (SELECT DISTINCT(ticker) FROM stocks_price WHERE volume = 0)")
db.commit()
db.close()

>> 1. DUMP cap >= 500000000000 & KOSPI
1.0. Delete old data
1.1. Update cap
1.1.1. Get already bid ticker
1.1.2. Get new data
1.2. Get ticker list
['000060', '000070', '000080', '000100', '000120', '000150', '000210', '000240', '000270', '000370', '000400', '000640', '000660', '000670', '000720', '000810', '000815', '000880', '000990', '001040', '001120', '001230', '001390', '001430', '001440', '001450', '001530', '001680', '001720', '001740', '001800', '001820', '001880', '002240', '002350', '002380', '002790', '002840', '003000', '003070', '003090', '003230', '003240', '003280', '003410', '003470', '003490', '003520', '003530', '003540', '003550', '003580', '003670', '003690', '003850', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005070', '005090', '005180', '005250', '005300', '005380', '005385', '005387', '005390', '005440', '005490', '005610', '005690', '005830', '005850', '005880', '005930', '005935', '005940', '006040', '006120', '006260', '006280', '0

In [6]:
tickers

[]

# 볼린저밴드 초기세팅

In [6]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 2. 볼린저밴드 초기세팅
print(">> 2. Init bollinger band")

# 2.0. 기존 데이터 제거
print("2.0. Delete old data")
cursor.execute("TRUNCATE boll")
db.commit()

# 2.1. 티커리스트 조회
print("2.1. Get ticker list")
cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
tickers = list(map(lambda x: x[0], cursor.fetchall()))

# 2.2. 기간 설정
print("2.2. Set period")
start = '20210601'
end = str(datetime.now().date()).replace('-', '')
print("period: (%s - %s)" %(start, end))

# 2.3. 새로운 데이터 업데이트
print("2.3. Update new data (%s - %s)" %(start, end))
day = 20
for ticker in tickers:
  try:
    # Get ticker's price
    cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['period'] = day
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = round(avg - std * 2)
      me = round(avg)
      hi = round(avg + std * 2)
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1)
    pass

>> 2. Init bollinger band
2.0. Delete old data
2.1. Get ticker list
2.2. Set period
period: (20210601 - 20211105)
2.3. Update new data (20210601 - 20211105)


# MFI 초기세팅

In [8]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

try:
  db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
  cursor = db.cursor()

  ### 3. MFI 초기세팅
  print(">> 3. Init MFI")

  # 3.0. 기존 데이터 제거
  print("3.0. Delete old data")
  cursor.execute("TRUNCATE mfi")
  db.commit()

  # 3.1. 티커리스트 조회
  print("3.1. Get ticker list")
  cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
  tickers = list(map(lambda x: x[0], cursor.fetchall()))

  # 3.2. 기간 설정
  print("3.2. Set period")
  start = '20210601'
  end = str(datetime.now().date()).replace('-', '')
  print("period: (%s - %s)" %(start, end))

  # 3.3. 새로운 데이터 업데이트
  print("3.3. Update new data (%s - %s)" %(start, end))
  day = 10
  print(1)
  for ticker in tickers:
    print(ticker)
    try:
      # Get ticker's price
      cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker + "' ORDER BY date DESC")
      priceDf = pd.DataFrame(cursor.fetchall())
      priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
      # Set Dataframe
      priceDf['period'] = day
      priceDf['tp'] = 0
      priceDf['mfi'] = 0.0
      priceDf['mfi_diff'] = 0.0
      # Set typical price
      for idx in range(0, len(priceDf)):
        row = priceDf.iloc[idx]
        priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
      # Set MFI
      for idx in range(0, len(priceDf)):
        if idx + day > len(priceDf) - 1:
          continue
        positiveRMF = 0
        negativeRMF = 0          
        for i in range(idx, idx + day):
          today = priceDf.iloc[i]
          yesterday = priceDf.iloc[i + 1]
          if today['tp'] > yesterday['tp']:
            positiveRMF += today['tp'] * today['volume']
          elif today['tp'] < yesterday['tp']:
            negativeRMF += today['tp'] * today['volume']
        MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
        priceDf.at[idx, 'mfi'] = MFI
        if idx > 0:
          priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
      mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
      conn = db_connection.connect()
      mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
      conn.close()
    except Exception as ex1:
      print('ex1', ex1, mfiDf)
      pass
  db.close()
except Exception as ex2:
  print('ex2', ex2)
  pass

>> 3. Init MFI
3.0. Delete old data
3.1. Get ticker list
3.2. Set period
period: (20210601 - 20211105)
3.3. Update new data (20210601 - 20211105)
1
000060
000070
000080
000100
000120
000150
000210
000240
000270
000370
000400
000640
000660
000670
000720
000810
000815
000880
000990
001040
001120
001230
001390
001430
001440
001450
001530
001680
001720
001740
001800
001820
001880
002240
002350
002380
002790
002840
003000
003070
003090
003230
003240
003410
003470
003490
003520
003530
003540
003550
003580
003670
003690
003850
004000
004020
004170
004370
004490
004800
004990
005070
005090
005180
005250
005300
005380
005385
005387
005390
005440
005490
005610
005690
005830
005850
005880
005930
005935
005940
006040
006120
006260
006280
006360
006390
006400
006405
006650
006800
00680K
007070
007310
007340
007570
007690
007700
008490
008560
008730
008770
008930
009150
009240
009420
009450
009540
009830
009900
009970
010050
010060
010120
010130
010620
010780
010950
011000
011070
011170
011200
01121

In [4]:
tickers

[]